# Quick adaptation of notes about FTS analysis from july 6th 2017.

Explicitly for github repository https://github.com/miramliu/FTS-Simulation.git

#### Sections: 

Design and construction of simulation

Realtime software

Analysis Equations

Time Constant Software

Calibration and Consistency

Mira 10/08/19

### Design and construction of simulation

*the nature of the interforograms it generates* - Mira

The interferograms generated by the FTS are convolved with a time constant due to the bolometer. It is the interferogram of light emitted from a blackbody that passes through into the detector cryostat. The amplifying electronics provides both an AC and DC output for the signal and a monitor of the detector bias. The interferogram center is not perfectly known because of the asynchronous sampling rate. Therefore the resulting interferogram is modulated by a carrier wave, decays over time, and is not centered.

*modeled properties* - Mira

The perfect interferogram should be a cosine wave multiplied by a Gaussian. This interferogram is also shifted to be centered at any point, optimally zero, hence $z_0$.

Therefore the function for the model deconvolved interferogram  is given by $$\psi_0 = \frac{1}{\sqrt{2\pi} p}cos(2\pi q(z-z_0))e^\frac{-(z-z_0)^2}{2p^2}$$

$\psi_0$: the output voltage

$z$: the optical delay

$z_0$: determines the position of the center (within a sample)

$p$: determines the width of the Gaussian

$q$: determines the frequency of oscilation

This perfect interferogram is then convolved by an RC circuit ($k_v$) and an AC coupling function ($k_{cc}$), both applied to the fourier transform of the model deconvolved interferogram $\psi_0$. $$k_v = \frac{1}{\frac{1}{\tau} + i(2\pi f)}, k_{cc} = \frac{2\pi f(.599)}{1 + i2\pi(.599)}$$ This leads to a final model of a convolved interferogram $$\psi_1 = \mathfrak{F}^{-1}[\mathfrak{F}[\psi_0]2\pi k_vk_{cc}]$$ where $\mathfrak{F}$ and $\mathfrak{F}^{-1}$ represent a fourier transform and an inverse fourier transform respectively.


### Realtime software

*commands and what inputs are used* - Mira

Using labjack and zaber, the appropriate serial port connected to the stage is chosen. The user is requested to set the maximum displacement, speed during movement, sample frequency, acceleration, sampling range, and appropriate voltage range wanted. The FTS runs with user selected number of iterations, velocity, and maximum delay.

The file is saved with the desired title, generally used to describe the parameters for a given run, as well as the date and time the run began.

### Analysis Equations

*Spectra from interferogram* - Mira

To get the most accurate spectrum from an interferogram the actual inteferogram is fourier transformed.

First a group of $2^n$ samples is selected that is centered around the white light fringe within one sample. This 'centering' of the white light fringe is first approximated by selecting the range of interferogram values that have the greatest variability in the smallest range. This will separate the white light fringe from the noise, which will have smaller variability in a larger range. Centering then ensures that the edges of the samples requested are relatively flat and mainly noise, while the center is the white light fringe.

This selection of samples is then multiplied by a Hanning function $(H)$ to select the desired series of discrete samples, and then fourier transformed. The Hanning function reduces aliasing, but at the cost of slightly reduced resolution.

In k-space it is then divided by a low-pass filter function, $$\sigma = \frac{n}{2\pi \tau}$$ which filters out high frequencies. It is also multiplied by a Gaussian $g$ that the signal is modulated by. $$g = \frac{1}{\sigma\sqrt{\pi}}e^\frac{-f^2}{\sigma^2}$$ This Gaussian compensates for the high gain at high frequencies from dividing by the low-pass filter function $\sigma$. In addition, the fourier transform of the actual interferogram is divided by the convolution kernel $$k_v = \frac{1}{\frac{1}{\tau} + i(2\pi f)}$$ taking into account the delay in the circuit. This leads to the equation of the deconvolved spectrum, shown below.

Deconvolved spectrum: $$\frac{\frac{1}{\sigma\sqrt{\pi}}e^\frac{-f^2}{\sigma^2}\mathfrak{F}[(H)I(t)]}{\frac{1}{\frac{1}{\tau} + i(2\pi f)} \tau}$$

This is the fourier transform of the interferogram multiplied by the gaussian and divided by the convolution kernel times the time constant. It is adjusting the signal in k-space to yield a more accurate final output.
This deconvolved spectrum is then inverse fourier transformed to produce the deconvolved interferogram.

Deconvolved interferogram: $$\mathfrak{F}^{-1}[\frac{\frac{1}{\sigma\sqrt{\pi}}e^\frac{-f^2}{\sigma^2}\mathfrak{F}[(H)I(t)]}{\frac{1}{\frac{1}{\tau} + i(2\pi f)} \tau}]$$


### Time Constant Software
*Time Constant and White Light Fringe* - Mira

The time constant in the convolution kernel was found using a Markov Chain Monte Carlo (MCMC) with several parameters. The model of the *convoluted* inteferogram is guessed to be the  model interferogram $\psi_0$ that is convolved with both a time constant kernel, whose time constant is $\tau$ , as well as a kernel from AC coupling whose time constant value $\tau_2$ is known. This model is then matched the the actual data and the MCMC is used to find the minimum $\chi^2$

Essentially the deconvolved interferogram, $$\mathfrak{F}^{-1}[\frac{\frac{1}{\sigma\sqrt{\pi}}e^\frac{-f^2}{\sigma^2}\mathfrak{F}[(H)I(t)]}{\frac{1}{\frac{1}{\tau} + i(2\pi f)} \tau}]$$ is compared to $$\psi_0 = A[\frac{1}{\sqrt{2\pi} p}cos(2\pi q(z-z_0))e^\frac{-(z-z_0)^2}{2p^2}]$$ for a range of values for $\tau$ , *p* and $z_0$ , *q* , and A (the amplitude).

The five free parameters in the model function are restrained to their reasonable ranges, each is given 250 walkers and 7000 steps. The first 2000 steps are discarded and the resulting histograms are used to find the values of the parameters that minimize $\chi^2$. 

It must be noted that the time constant is only explicitly relevant at high speeds, and depends on the detector used. 

After the interferogram has been convolved, it is demodulated. This is done by shifting the spectrum in k-space such that the peak of the carrier wave is centered at zero. This is done by using the maximum y value and its corresponding x value and shifting it accordingly. The spectrum cyclically rotated about zero, and multiplied by a hanning function to remove the carrier wave. The two remaining lobes are then fourier transformed and cyclically rotated again resulting in a demodulated and deconvoluted interferogram. The white light fringe then ends up at the center, where the mirror is perfectly at the center.

This white light fringe can be found, within a sample, by comparing the peaks of the demodulated and deconvoluted interferogram.  

### Calibration and Consistency
*Calibration software* - Mira

Using a chopper at different known frequencies and window sized we calibrate the instrument. This data is used to find the carrier wave and to test the accuracy of the resulting spectrum. 

*consistency checks* - Mira

The software converts the data into a function of time instead of delay to get an accurate deconvolution. 